<a href="https://colab.research.google.com/github/Lucasfontanelli/atividademachinelearning/blob/main/trabalho_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/creditcard - menor balanceado.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,82450.0,1.314539,0.590643,-0.666593,0.716564,0.301978,-1.125467,0.388881,-0.288390,-0.132137,...,-0.170307,-0.429655,-0.141341,-0.200195,0.639491,0.399476,-0.034321,0.031692,0.76,0
1,50554.0,-0.798672,1.185093,0.904547,0.694584,0.219041,-0.319295,0.495236,0.139269,-0.760214,...,0.202287,0.578699,-0.092245,0.013723,-0.246466,-0.380057,-0.396030,-0.112901,4.18,0
2,55125.0,-0.391128,-0.245540,1.122074,-1.308725,-0.639891,0.008678,-0.701304,-0.027315,-2.628854,...,-0.133485,0.117403,-0.191748,-0.488642,-0.309774,0.008100,0.163716,0.239582,15.00,0
3,116572.0,-0.060302,1.065093,-0.987421,-0.029567,0.176376,-1.348539,0.775644,0.134843,-0.149734,...,0.355576,0.907570,-0.018454,-0.126269,-0.339923,-0.150285,-0.023634,0.042330,57.00,0
4,90434.0,1.848433,0.373364,0.269272,3.866438,0.088062,0.970447,-0.721945,0.235983,0.683491,...,0.103563,0.620954,0.197077,0.692392,-0.206530,-0.021328,-0.019823,-0.042682,0.00,0


# Seleção de Atributos (Feature Selection)


In [ ]:
#SelectKBest
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif

df = pd.read_csv('/content/creditcard - menor balanceado.csv')


k = 10
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)


selected_features = selector.get_support(indices=True)
print(X.columns[selected_features])

Index(['V3', 'V4', 'V7', 'V9', 'V10', 'V11', 'V12', 'V14', 'V16', 'V17'], dtype='object')


In [ ]:
#Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
selector = RFE(model, n_features_to_select=10)
selector = selector.fit(X, y)


selected_features = X.columns[selector.support_]
print(selected_features)

Selected features with RFE: Index(['V3', 'V4', 'V7', 'V10', 'V11', 'V12', 'V14', 'V16', 'V17', 'V19'], dtype='object')


In [ ]:
#Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X, y)


feature_importances = pd.Series(model.feature_importances_, index=X.columns)
top_features = feature_importances.nlargest(10)

print(top_features.index)

Top features based on importance: Index(['V14', 'V10', 'V12', 'V17', 'V4', 'V3', 'V16', 'V11', 'V2', 'V9'], dtype='object')


#Técnicas de Balanceamento (Sampling)

In [ ]:
#Oversampling com SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv('/content/creditcard - menor balanceado.csv')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)


y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Resultados com SMOTE:
[[370   7]
 [ 21 130]]
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       377
           1       0.95      0.86      0.90       151

    accuracy                           0.95       528
   macro avg       0.95      0.92      0.93       528
weighted avg       0.95      0.95      0.95       528



In [ ]:
#Undersampling da Classe Majoritária
from imblearn.under_sampling import RandomUnderSampler


undersample = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersample.fit_resample(X_train, y_train)


model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)


y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Resultados com Undersampling:
[[372   5]
 [ 16 135]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       377
           1       0.96      0.89      0.93       151

    accuracy                           0.96       528
   macro avg       0.96      0.94      0.95       528
weighted avg       0.96      0.96      0.96       528



#Tuning de Hiperparâmetros

In [ ]:
#Random Forest com GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(random_state=42)


param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}


grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='f1', n_jobs=-1)
grid_search_rf.fit(X_train_resampled, y_train_resampled)


best_rf = grid_search_rf.best_estimator_
print(grid_search_rf.best_params_)


In [ ]:
#SVM com RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC


svm = SVC(random_state=42)


param_dist_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}


random_search_svm = RandomizedSearchCV(estimator=svm, param_distributions=param_dist_svm, cv=5, scoring='f1', n_iter=10, random_state=42, n_jobs=-1)
random_search_svm.fit(X_train_resampled, y_train_resampled)

best_svm = random_search_svm.best_estimator_
print(random_search_svm.best_params_)

{'kernel': 'linear', 'gamma': 1, 'C': 10}


In [ ]:
#XGBoost com GridSearchCV
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')


param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 1.0],
    'scale_pos_weight': [1, 5, 10]
}


grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, cv=5, scoring='f1', n_jobs=-1)
grid_search_xgb.fit(X_train_resampled, y_train_resampled)


best_xgb = grid_search_xgb.best_estimator_
print(grid_search_xgb.best_params_)

In [ ]:
 #Modelos Otimizados
 from sklearn.metrics import classification_report, confusion_matrix


y_pred_rf = best_rf.predict(X_test)
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))




[[371   6]
 [ 15 136]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       377
           1       0.96      0.90      0.93       151

    accuracy                           0.96       528
   macro avg       0.96      0.94      0.95       528
weighted avg       0.96      0.96      0.96       528



#Ensemble de Modelos

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

dt = DecisionTreeClassifier(random_state=42)
svm = SVC(probability=True, random_state=42)
knn = KNeighborsClassifier()


voting_clf_hard = VotingClassifier(estimators=[('dt', dt), ('svm', svm), ('knn', knn)], voting='hard')
voting_clf_soft = VotingClassifier(estimators=[('dt', dt), ('svm', svm), ('knn', knn)], voting='soft')


voting_clf_hard.fit(X_train_resampled, y_train_resampled)
y_pred_hard = voting_clf_hard.predict(X_test)
print("Resultados do Voting Classifier (hard):")
print(confusion_matrix(y_test, y_pred_hard))
print(classification_report(y_test, y_pred_hard))


voting_clf_soft.fit(X_train_resampled, y_train_resampled)
y_pred_soft = voting_clf_soft.predict(X_test)
print("Resultados do Voting Classifier (soft):")
print(confusion_matrix(y_test, y_pred_soft))
print(classification_report(y_test, y_pred_soft))


Resultados do Voting Classifier (hard):
[[275 102]
 [ 44 107]]
              precision    recall  f1-score   support

           0       0.86      0.73      0.79       377
           1       0.51      0.71      0.59       151

    accuracy                           0.72       528
   macro avg       0.69      0.72      0.69       528
weighted avg       0.76      0.72      0.73       528

Resultados do Voting Classifier (soft):
[[328  49]
 [ 26 125]]
              precision    recall  f1-score   support

           0       0.93      0.87      0.90       377
           1       0.72      0.83      0.77       151

    accuracy                           0.86       528
   macro avg       0.82      0.85      0.83       528
weighted avg       0.87      0.86      0.86       528



In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)


voting_clf_ensemble = VotingClassifier(estimators=[('rf', rf), ('gb', gb)], voting='soft')
voting_clf_ensemble.fit(X_train_resampled, y_train_resampled)


y_pred_ensemble = voting_clf_ensemble.predict(X_test)
print("Resultados do Voting Classifier (Random Forest + Gradient Boosting):")
print(confusion_matrix(y_test, y_pred_ensemble))
print(classification_report(y_test, y_pred_ensemble))

Resultados do Voting Classifier (Random Forest + Gradient Boosting):
[[366  11]
 [ 13 138]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       377
           1       0.93      0.91      0.92       151

    accuracy                           0.95       528
   macro avg       0.95      0.94      0.94       528
weighted avg       0.95      0.95      0.95       528



#Avaliação das Métricas

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report


y_pred_final = voting_clf_ensemble.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_final)
print("Acurácia:", accuracy)

f1_macro = f1_score(y_test, y_pred_final, average='macro')
print("F1-Score Macro:", f1_macro)

conf_matrix = confusion_matrix(y_test, y_pred_final)
print("Matriz de Confusão:\n", conf_matrix)


print("Relatório de Classificação:\n", classification_report(y_test, y_pred_final))
